In [56]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable
from chainer import optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [70]:
# Set data
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data.astype(np.float32)
Y = iris.target.astype(np.float32)
N = Y.size
Y2 = np.zeros(3 * N).reshape(N,3).astype(np.float32)
for i in range(N):
    Y2[i,np.int(Y[i])] = np.float32(1.0)

index = np.arange(N)
xtrain = X[index[index % 2 != 0],:]
ytrain = Y2[index[index % 2 != 0],:]
ytrain_ans = Y[index[index % 2 != 0]]
xtest = X[index[index % 2 == 0],:]
yans = Y[index[index % 2 == 0]]

In [58]:
# Define model

class MyAE(Chain):
    def __init__(self):
        super(MyAE, self).__init__(
            l1=L.Linear(4,2),
            l2=L.Linear(2,4),
        )

    def __call__(self,x):
        bv = self.fwd(x)
        return F.mean_squared_error(bv, x)

    def fwd(self,x):
        fv = F.sigmoid(self.l1(x))
        bv = self.l2(fv)
        return bv

In [59]:
# Initialize model

ae_model = MyAE()
optimizer = optimizers.SGD()
optimizer.setup(ae_model)

In [60]:
# Learn

n = 75
bs = 30
for j in range(3000):
    sffindx = np.random.permutation(range(n))
    for i in range(0, n, bs):
        x = Variable(xtrain[sffindx[i:(i+bs) if (i+bs) < n else n]])
        ae_model.zerograds()
        loss = ae_model(x)
        loss.backward()
        optimizer.update()

In [93]:
# Result
x = Variable(xtrain, volatile='on')
yt = F.sigmoid(ae_model.l1(x))
ans = yt.data
#for i in range(n):
#    print (i, ans[i,:])

In [87]:
# 事項符号化のデータでロジクスティック回帰
# Define model
class IrisRogi(Chain):
    def __init__(self):
        super(IrisRogi, self).__init__(
            l1=L.Linear(2,3),
        )

    def __call__(self,x,y):
        return F.mean_squared_error(self.fwd(x), y)

    def fwd(self,x):
        return F.softmax(self.l1(x))

# Initialize model

rg_model = IrisRogi()
optimizer = optimizers.Adam()
optimizer.setup(rg_model)

In [89]:
n = 75
bs = 25
for j in range(5000):
    sffindx = np.random.permutation(n)
    for i in range(0, n, bs):
        x = Variable(xtrain_ae.data[sffindx[i:(i+bs) if (i+bs) < n else n]])
        y = Variable(ytrain[sffindx[i:(i+bs) if (i+bs) < n else n]])
        rg_model.zerograds()
        loss = rg_model(x,y)
        loss.backward()
        optimizer.update()

In [91]:
# train Result
xtrain_ae = F.sigmoid(ae_model.l1(Variable(xtrain, volatile='on')))
yy = rg_model.fwd(xtrain_ae)
ans = yy.data
nrow, ncol = ans.shape
ok = 0
for i in range(nrow):
    cls = np.argmax(ans[i,:])
    # print (ans[i,:], cls)
    if cls == ytrain_ans[i]:
        ok += 1

print( ok, "/", nrow, " = ", (ok * 1.0)/nrow)

73 / 75  =  0.9733333333333334


In [92]:
# test Result
xtest_ae = F.sigmoid(ae_model.l1(Variable(xtest, volatile='on')))
yy = rg_model.fwd(xtest_ae)
ans = yy.data
nrow, ncol = ans.shape
ok = 0
for i in range(nrow):
    cls = np.argmax(ans[i,:])
    # print (ans[i,:], cls)
    if cls == yans[i]:
        ok += 1

print( ok, "/", nrow, " = ", (ok * 1.0)/nrow)

72 / 75  =  0.96
